In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.1 MB/s 
     |████████████████████████████████| 6.6 MB 36.8 MB/s 
     |████████████████████████████████| 101 kB 12.1 MB/s 
     |████████████████████████████████| 596 kB 74.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer
import sklearn
from sklearn.metrics import confusion_matrix

In [ ]:
df = pd.read_csv('train.csv')
len(df)

120000

In [ ]:
df.drop(df.tail(100000).index,
        inplace = True)

In [ ]:
df.head()

,Category,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [ ]:
dt = pd.read_csv('test.csv')
len(dt)

7600

In [ ]:
dt.drop(dt.tail(5500).index,
        inplace = True)

In [ ]:
dt.head()

,Category,Title,Description
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...


In [ ]:
df.info()
print()
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Category     20000 non-null  int64 
 1   Title        20000 non-null  object
 2   Description  20000 non-null  object
dtypes: int64(1), object(2)
memory usage: 625.0+ KB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2100 entries, 0 to 2099
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Category     2100 non-null   int64 
 1   Title        2100 non-null   object
 2   Description  2100 non-null   object
dtypes: int64(1), object(2)
memory usage: 65.6+ KB


### **Data Preparation**

In [ ]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
df['Category_target']= label_encoder.fit_transform(df['Category']) 
df.head(10)

,Category,Title,Description,Category_target
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",2
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,2
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,2
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,2
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",2
5,3,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...,2
6,3,Money Funds Fell in Latest Week (AP),AP - Assets of the nation's retail money marke...,2
7,3,Fed minutes show dissent over inflation (USATO...,USATODAY.com - Retail sales bounced back a bit...,2
8,3,Safety Net (Forbes.com),Forbes.com - After earning a PH.D. in Sociolog...,2
9,3,Wall St. Bears Claw Back Into the Black,"NEW YORK (Reuters) - Short-sellers, Wall Stre...",2


In [ ]:
dt['Category_target']= label_encoder.fit_transform(dt['Category']) 
dt.head(10)

,Category,Title,Description,Category_target
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...,2
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o...",3
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...,3
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...,3
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...,3
5,4,Open Letter Against British Copyright Indoctri...,The British Department for Education and Skill...,3
6,4,Loosing the War on Terrorism,"\\""Sven Jaschan, self-confessed author of the ...",3
7,4,"FOAFKey: FOAF, PGP, Key Distribution, and Bloo...",\\FOAF/LOAF and bloom filters have a lot of i...,3
8,4,E-mail scam targets police chief,"Wiltshire Police warns about ""phishing"" after ...",3
9,4,"Card fraud unit nets 36,000 cards","In its first two years, the UK's dedicated car...",3


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
token = tokenizer.encode_plus(
    df['Description'].iloc[0], 
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

In [ ]:
token.input_ids

<tf.Tensor: shape=(1, 256), dtype=int32, numpy=
array([[  101, 11336, 27603,   118,  6373,   118, 18275,  1116,   117,
         6250,  1715,   112,   188,   173, 11129,  1979,   165,  1467,
         1104, 18737,   118,   172,  5730,  4724,   117,  1132,  3195,
         2448,  1254,   119,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [ ]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

X_test_input_ids = np.zeros((len(dt), 256))
X_test_attn_masks = np.zeros((len(dt), 256))

In [ ]:
def generate_encoded_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['Description'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [ ]:
X_input_ids, X_attn_masks = generate_encoded_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [ ]:
X_test_input_ids, X_test_attn_masks = generate_encoded_data(dt, X_test_input_ids, X_test_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [ ]:
labels = np.zeros((len(df), 5))
labels.shape

(20000, 5)

In [ ]:
labels_test = np.zeros((len(dt), 5))
labels_test.shape

(2100, 5)

In [ ]:
labels[np.arange(len(df)), df['Category_target'].values] = 1 # one-hot encoded target tensor
labels_test[np.arange(len(dt)), dt['Category_target'].values] = 1 

In [ ]:
labels

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [ ]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [ ]:
dataset_test = tf.data.Dataset.from_tensor_slices((X_test_input_ids, X_test_attn_masks, labels_test))
dataset_test.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [ ]:
def DatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [ ]:
dataset = dataset.map(DatasetMapFunction) # converting to required format for tensorflow dataset 

In [ ]:
dataset_test = dataset_test.map(DatasetMapFunction)

In [ ]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [ ]:
dataset_test.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [ ]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor

In [ ]:
dataset_test = dataset_test.batch(16, drop_remainder=True)

In [ ]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [ ]:
dataset_test.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [ ]:
train_size = int((len(df)//16)) # for each 16 batch of data we will have len(df)//16 samples
test_size = int(len(dt))

In [ ]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)
test_dataset = dataset_test.take(test_size)
#val_test_dataset = dataset_test.skip(test_size)

In [ ]:
len(train_dataset)

1250

In [ ]:
len(test_dataset)

131

### **Model**

In [ ]:
from transformers import TFBertModel

In [ ]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

text_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
text_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [ ]:
text_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [ ]:
hist = text_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2
)

Epoch 1/2
1250/1250 [==============================] - 1033s 818ms/step - loss: 0.3644 - accuracy: 0.8734
Epoch 2/2
1250/1250 [==============================] - 1037s 830ms/step - loss: 0.1869 - accuracy: 0.9367


In [ ]:
text_model.save('text_model')

INFO:tensorflow:Assets written to: text_model/assets


INFO:tensorflow:Assets written to: text_model/assets


### **Prediction**

In [ ]:
print(test_dataset)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>


In [ ]:
len(test_dataset)

131

In [ ]:
predicted = text_model.predict(test_dataset)
predicted

array([[5.9627979e-03, 2.9523185e-04, 9.7139663e-01, 2.2254849e-02,
        9.0610447e-05],
       [3.2149688e-03, 2.7823594e-04, 1.4698720e-03, 9.9490911e-01,
        1.2787683e-04],
       [1.5164125e-02, 7.2296680e-04, 2.9031718e-02, 9.5494580e-01,
        1.3532901e-04],
       ...,
       [3.3184297e-03, 5.6395115e-04, 5.9871938e-02, 9.3611199e-01,
        1.3385572e-04],
       [2.3923263e-01, 2.5613043e-03, 1.0078588e-02, 7.4770880e-01,
        4.1861041e-04],
       [1.1602710e-02, 8.6761825e-04, 5.8538800e-01, 4.0196615e-01,
        1.7546865e-04]], dtype=float32)

In [ ]:
len(predicted)

2096

In [ ]:
pred_class = np.argmax(predicted,axis=1)
pred_class

array([2, 3, 3, ..., 3, 3, 2])

In [ ]:
len(pred_class)

2096

In [ ]:
n = len(dt['Category_target'])
a = n%16
if(a!=0):
  dt.drop(dt.tail(a).index,
        inplace = True)
dt['Category_target']

0       2
1       3
2       3
3       3
4       3
       ..
2091    2
2092    0
2093    3
2094    3
2095    3
Name: Category_target, Length: 2096, dtype: int64

In [ ]:
dt.tail()

,Category,Title,Description,Category_target
2091,3,Oracle v. Peoplesoft: the joke is on...,Opinion I thought it was a joke when Oracle fi...,2
2092,1,Iran May Soon Resume Uranium Enrichment (AP),"AP - Iran may resume uranium enrichment ""any m...",0
2093,4,Media center at your fingertips,Apple #39;s splashy digital music player has e...,3
2094,4,The battle for DR Congo's wildlife,As donors pledge \$40m to save DR Congo's wild...,3
2095,4,Porn Processor Settles Deceptive-Billing Charges,WASHINGTON (Reuters) - A pornography bill-proc...,3


In [ ]:
print(f"Accuracy is {sklearn.metrics.accuracy_score(dt['Category_target'], pred_class)}")

Accuracy is 0.9255725190839694


In [ ]:
text_model = tf.keras.models.load_model('text_model')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['World','Sport','Business', 'Tech']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [ ]:
input_text = input('Enter test text here: ')
processed_data = prepare_data(input_text, tokenizer)
#print(processed_data)
result = make_prediction(text_model, processed_data=processed_data)
print(f"Predicted result: {result}")

Enter test text here: elon
Predicted result: Tech
